In [21]:
import json

In [1]:
shape_exemplo = ShapeFile.objects.get(name="Propriedades", companies__name="GES - UHE-ITA")

In [25]:
print(json.dumps(shape_exemplo.metadata['fill_color'], indent=4))

"rgba(255,255,255,0)"


In [10]:
companies = Company.objects.filter(metadata__company_prefix__in=["UHSO",
"UHCB",
"UHIT",
"UHMA",
"UHPF",
"UHPP",
"UHSA",
"UHJA",
"UHMI",
"UHSS",], name__startswith="GES")

In [15]:
areas = ShapeFile.objects.filter(name="Áreas").first()

In [18]:
areas.uuid = None
areas.save()
areas.companies.set(Company.objects.filter(name="GES - UHE-ITA"))

In [26]:
for company in companies:
    shape_group = ShapeFile.objects.get(companies=company, name="Áreas")
    shape_file = ShapeFile(
        name="RESERVATÓRIO NA COTA MAXIMO OPERACIONAL NORMAL",
        parent=shape_group,
        metadata={
            "sync": {
                "url": "https://gis.engieenergia.com.br/gis/rest/services/Patrimonial/UHES_RESNOR/MapServer/1",
                "auth": {
                    "url": "https://gis.engieenergia.com.br/portal/sharing/rest/generateToken",
                    "type": "arcgis_token",
                    "password": "int3gr@cao#gis&hid",
                    "username": "integracao.hidros"
                },
                "type": "arcgis",
                "params": [
                    {
                        "name": "where",
                        "value": "ATIVO='{}'".format(company.metadata['company_prefix'])
                    },
                    {
                        "name": "outFields",
                        "value": "*"
                    },
                    {
                        "name": "spatialRel",
                        "value": "esriSpatialRelIntersects"
                    },
                    {
                        "name": "returnGeometry",
                        "value": "true"
                    },
                    {
                        "name": "outSR",
                        "value": "4326"
                    },
                    {
                        "name": "featureEncoding",
                        "value": "esriDefault"
                    },
                    {
                        "name": "f",
                        "value": "geojson"
                    }
                ]
            },
            "fill_color": "rgba(255,255,255,0)",
            "stroke_color": "rgb(85, 37, 121)",
        }
    )
    shape_file.save()
    shape_file.companies.add(company)

/Users/marcos/Library/Caches/pypoetry/virtualenvs/apps-e-KIzLL9-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ShapeFile.synced_at received a naive datetime (2023-09-27 15:58:15.037291) while time zone support is active.
  RuntimeWarning)
/Users/marcos/Library/Caches/pypoetry/virtualenvs/apps-e-KIzLL9-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ShapeFile.synced_at received a naive datetime (2023-09-27 15:58:15.214825) while time zone support is active.
  RuntimeWarning)
/Users/marcos/Library/Caches/pypoetry/virtualenvs/apps-e-KIzLL9-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ShapeFile.synced_at received a naive datetime (2023-09-27 15:58:15.349912) while time zone support is active.
  RuntimeWarning)
/Users/marcos/Library/Caches/pypoetry/virtualenvs/apps-e-KIzLL9-py3.7/lib/python3.7/site-packages/django/d

In [5]:
from apps.maps.asynchronous import execute_arcgis_sync
from tqdm.notebook import tqdm
# from arcgis.gis import GIS

In [6]:
for shape_file in tqdm(ShapeFile.objects.filter(
    name="RESERVATÓRIO NA COTA MAXIMO OPERACIONAL NORMAL"
)):
    execute_arcgis_sync(shape_file.uuid)

  0%|          | 0/10 [00:00<?, ?it/s]

SSLError: HTTPSConnectionPool(host='gis.engieenergia.com.br', port=443): Max retries exceeded with url: /portal/sharing/rest/generateToken (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1091)')))

In [4]:
import requests

concla = requests.get('https://cnae.ibge.gov.br/?view=secao&tipo=cnae&versaosubclasse=10&versaoclasse=7&secao=A',verify=False)

print(concla.status_code)

SSLError: HTTPSConnectionPool(host='cnae.ibge.gov.br', port=443): Max retries exceeded with url: /?view=secao&tipo=cnae&versaosubclasse=10&versaoclasse=7&secao=A (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1091)')))

In [7]:
import requests
import urllib3
import ssl


class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [10]:
teste=get_legacy_session().get('https://gis.engieenergia.com.br/gis/rest/services/Patrimonial/UHES_RESNOR/MapServer/1')

In [12]:
teste.__dict__

{'_content': b'<!DOCTYPE html>\n<html>\n<head>\n  <script src="login/config/dojo.js"></script>\n  <script src="jsapi/dojo/dojo.js"></script>\n  <script>\n    require(["login", "dojo/domReady!"], function(login) {\n      login.run();\n    });\n  </script>\n</head>\n<body>\n</body>\n</html>\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Cache-Control': 'private', 'Content-Type': 'text/html', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Last-Modified': 'Sat, 16 Oct 2021 07:03:30 GMT', 'Accept-Ranges': 'bytes', 'ETag': 'W/"262-1634367810000"', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'Date': 'Wed, 04 Oct 2023 12:47:16 GMT', 'Content-Length': '262', 'Set-Cookie': 'TS010cb77d=01b40ee4a3d2a69302ee2a4fb8294e94c4167a14586dc972a492f3633bc827b6982b8a40b91735ab208a08288bc83554501c08eaf2; Path=/; Domain=.gis.engieenergia.com.br'},
 'raw': <urllib3.response.HTTPRes